In [51]:
!pip install gensim

In [34]:
from sklearn.model_selection import train_test_split
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score

In [2]:
train_df = pd.read_csv('./development.csv', sep=',')
train_df

,sentiment,ids,date,flag,user,text
0,1,1833972543,Mon May 18 01:08:27 PDT 2009,NO_QUERY,Killandra,"@MissBianca76 Yes, talking helps a lot.. going..."
1,1,1980318193,Sun May 31 06:23:17 PDT 2009,NO_QUERY,IMlisacowan,SUNSHINE. livingg itttt. imma lie on the grass...
2,1,1994409198,Mon Jun 01 11:52:54 PDT 2009,NO_QUERY,yaseminx3,@PleaseBeMine Something for your iphone
3,0,1824749377,Sun May 17 02:45:34 PDT 2009,NO_QUERY,no_surprises,@GabrielSaporta couldn't get in to the after p...
4,0,2001199113,Tue Jun 02 00:08:07 PDT 2009,NO_QUERY,Rhi_ShortStack,@bradiewebbstack awww is andy being mean again...
...,...,...,...,...,...,...
224989,0,2261324310,Sat Jun 20 20:36:48 PDT 2009,NO_QUERY,CynthiaBuroughs,@Dropsofreign yeah I hope Iran people reach fr...
224990,1,1989408152,Mon Jun 01 01:25:45 PDT 2009,NO_QUERY,unitechy,Trying the qwerty keypad
224991,0,1991221316,Mon Jun 01 06:38:10 PDT 2009,NO_QUERY,Xaan,I love Jasper &amp; Jackson but that wig in th...
224992,0,2239702807,Fri Jun 19 08:51:56 PDT 2009,NO_QUERY,Ginger_Billie,I am really tired and bored and bleh! I feel c...


In [53]:
test_df = pd.read_csv('./evaluation.csv', sep=',')
test_df

,ids,date,flag,user,text
0,1999928613,Mon Jun 01 21:04:54 PDT 2009,NO_QUERY,urbanperspectiv,@engteacher I'm pretty much the same ...in eit...
1,1827919331,Sun May 17 11:43:09 PDT 2009,NO_QUERY,therealsecret,@JaxLicurse same here have a gr8 week ahead
2,1563640134,Sun Apr 19 23:37:52 PDT 2009,NO_QUERY,bitchville,@blacksweater that's just nightmares all over
3,2189786396,Tue Jun 16 00:54:27 PDT 2009,NO_QUERY,epi_longo,@dlead @jishanvn cháº¯c pháº£i Äi thi Äáº¡i ...
4,1974034022,Sat May 30 12:18:33 PDT 2009,NO_QUERY,Curiosafmmb,@CA_Sweetie awe. ok sweetie. ttyl. *hugs*
...,...,...,...,...,...
74994,1933009490,Tue May 26 22:45:42 PDT 2009,NO_QUERY,Irv25,"@NIYANA man down,code 10...I'm going 2 try &am..."
74995,1563759085,Mon Apr 20 00:08:25 PDT 2009,NO_QUERY,iHomeTech,@senhordaguerra Thank you for following me: I ...
74996,1754026098,Sun May 10 04:38:13 PDT 2009,NO_QUERY,danger_skies,Why isn't Friday's Hollyoaks available for dow...
74997,2288725776,Mon Jun 22 19:32:13 PDT 2009,NO_QUERY,tracyewilli,@hollywilli she WAS !!! she was a MADAM they q...


In [3]:
X = train_df.loc[:,['text', 'user', 'date']] #adding also the date
y = train_df.loc[:,['sentiment']]

In [4]:
X 

,text,user,date
0,"@MissBianca76 Yes, talking helps a lot.. going...",Killandra,Mon May 18 01:08:27 PDT 2009
1,SUNSHINE. livingg itttt. imma lie on the grass...,IMlisacowan,Sun May 31 06:23:17 PDT 2009
2,@PleaseBeMine Something for your iphone,yaseminx3,Mon Jun 01 11:52:54 PDT 2009
3,@GabrielSaporta couldn't get in to the after p...,no_surprises,Sun May 17 02:45:34 PDT 2009
4,@bradiewebbstack awww is andy being mean again...,Rhi_ShortStack,Tue Jun 02 00:08:07 PDT 2009
...,...,...,...
224989,@Dropsofreign yeah I hope Iran people reach fr...,CynthiaBuroughs,Sat Jun 20 20:36:48 PDT 2009
224990,Trying the qwerty keypad,unitechy,Mon Jun 01 01:25:45 PDT 2009
224991,I love Jasper &amp; Jackson but that wig in th...,Xaan,Mon Jun 01 06:38:10 PDT 2009
224992,I am really tired and bored and bleh! I feel c...,Ginger_Billie,Fri Jun 19 08:51:56 PDT 2009


In [5]:
import re


for index, row in X.iterrows():
    row['text'] = re.sub('@\S+', ' ', row['text']) #removing @ from names
    row['date'] = re.sub()

In [6]:
X

,text,user,date
0,"Yes, talking helps a lot.. going through it,...",Killandra,Mon May 18 01:08:27 PDT 2009
1,SUNSHINE. livingg itttt. imma lie on the grass...,IMlisacowan,Sun May 31 06:23:17 PDT 2009
2,Something for your iphone,yaseminx3,Mon Jun 01 11:52:54 PDT 2009
3,couldn't get in to the after party.,no_surprises,Sun May 17 02:45:34 PDT 2009
4,awww is andy being mean again? Now i want m...,Rhi_ShortStack,Tue Jun 02 00:08:07 PDT 2009
...,...,...,...
224989,yeah I hope Iran people reach freedom... the...,CynthiaBuroughs,Sat Jun 20 20:36:48 PDT 2009
224990,Trying the qwerty keypad,unitechy,Mon Jun 01 01:25:45 PDT 2009
224991,I love Jasper &amp; Jackson but that wig in th...,Xaan,Mon Jun 01 06:38:10 PDT 2009
224992,I am really tired and bored and bleh! I feel c...,Ginger_Billie,Fri Jun 19 08:51:56 PDT 2009


In [7]:
tfid = TfidfVectorizer(max_df= 0.40, min_df=0)#0.50 #0.80 , performa bene anche con 0.40 #piu' alzo min_df più funziona male #0.005 buono #provare con 0.001 e 0.0005 (0.00005  consegnato)(0.00001  consegnato 2)
X = tfid.fit_transform(X.text, X.user) 

In [8]:
X

<224994x88150 sparse matrix of type '<class 'numpy.float64'>'
	with 2591822 stored elements in Compressed Sparse Row format>

In [9]:
#try usng multiple features? add user name

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=77)

In [32]:
logistic_regression = LogisticRegression(penalty='l2', random_state=42, C=4,fit_intercept= True, max_iter= 500, solver= 'liblinear')

In [33]:
cv_scores = cross_val_score(logistic_regression, X_train, y_train, cv=5, scoring='f1_macro', n_jobs=-1)


mean = cv_scores.mean()
std = cv_scores.std() * 2
print(f"f1_macro: {mean:.3f} (+/- {std:.3f})")

f1_macro: 0.779 (+/- 0.006)


In [13]:
#logistic_regression.fit(X_train, y_train)

In [14]:
#y_pred = logistic_regression.predict(X_test)

In [15]:
#y_pred

In [16]:
#y_test = list(y_test['sentiment'])

In [17]:
#y_test #printing list of our test sentiment values

In [18]:
#y_pred = list(y_pred)

In [19]:
#y_pred #printing list of our predictions made

In [20]:
#Linear SVC

In [49]:
linear_svc = LinearSVC(random_state=77, penalty="l2")

#param_grid = {'loss': ['hinge', 'squared_hinge'], 'fit_intercept': [True, False], 'C': list(range(1,10,2))}

#gridSearch = GridSearchCV(linear_svc, param_grid, scoring='f1_weighted', cv=5, n_jobs=-1)

#gridSearch.fit(X_train, y_train)

#print(gridSearch.best_params_)

In [50]:
#Liline
cv_scores = cross_val_score(linear_svc, X_train, y_train, cv=5, scoring='f1_macro', n_jobs=-1)

mean = cv_scores.mean()
std = cv_scores.std() * 2
print(f"f1_macro: {mean:.3f} (+/- {std:.3f})")

f1_macro: 0.772 (+/- 0.002)


In [ ]:
#from sklearn.metrics import accuracy_score

In [ ]:
#accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(logistic_regression, X_train, y_train, cv=5, scoring='f1_macro', n_jobs=-1)


mean = cv_scores.mean()
std = cv_scores.std() * 2
print(f"f1_macro: {mean:.3f} (+/- {std:.3f})")

In [ ]:
#from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# neigh = KNeighborsClassifier(n_neighbors=3)

In [ ]:
# neigh.fit(X_train, y_train)

In [ ]:
# y_pred = neigh.predict(X_test)

In [ ]:
# accuracy_score(y_test, y_pred)

In [ ]:
#from sklearn.neural_network import MLPClassifier

In [ ]:
"""
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
"""

In [ ]:
#accuracy_score(y_test, y_pred)

In [ ]:
"""
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='f1_macro', n_jobs=-1)


mean = cv_scores.mean()
std = cv_scores.std() * 2
print(f"f1_macro: {mean:.3f} (+/- {std:.3f})
"""

In [ ]:
#from sklearn.ensemble import RandomForestClassifier

In [ ]:
"""
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
"""

In [ ]:
#accuracy_score(y_test, y_pred)

In [ ]:
mean = cv_scores.mean()
std = cv_scores.std() * 2
print(f"f1_macro: {mean:.3f} (+/- {std:.3f})")

In [ ]:
#Let me try using regression and rounding the results to the nearest value (since regression predicts continous varibales)

from sklearn.linear_model import LinearRegression

In [ ]:
clf = LinearRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
y_pred;

In [ ]:
y_pred = list(y_pred)

for i in range(len(y_pred)):
    
    
    if y_pred[i] < .5:
        y_pred[i] = 0
    else:
        y_pred[i] = 1

In [ ]:
y_pred;

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='f1_macro', n_jobs=-1)


mean = cv_scores.mean()
std = cv_scores.std() * 2
print(f"f1_macro: {mean:.3f} (+/- {std:.3f})")